In [1]:
from chemtbd.io import Agilent
import numpy as np
import pandas as pd

In [2]:
agi = Agilent.from_root('data')

In [3]:
def matchlib2area_one(key,libdf, areadf,areatype):
    returndf = libdf.copy()
    
    if areatype == 'tic':
        colname = 'tic_area'
    elif areatype == 'fid':
        colname = 'fid_area'
    else:
        print('incorrect area type')

    returndf[colname] = np.nan
    #find shortest
    if len(libdf) <= len(areadf):
        short = libdf
        long = areadf
    else:
        short = areadf
        long = libdf
        
    #loop and match
    match_list = []
    for index,row in short.iterrows():
        sqrError = (short['rt'].ix[index]-long['rt'].ix[index:len(long)])**2    
        area_row = sqrError.idxmin(axis=1)
        match_list.append(areadf['area'].ix[area_row])
    if areatype == 'tic':
        returndf['tic_area'] = match_list
    elif areatype == 'fid':
        returndf['fid_area'] = match_list
    
    return returndf

def matchlib2area_all(data_dic,areatype='tic'):
    libdf = data_dic.results('lib').groupby('key')
    areadf  = data_dic.results(areatype).groupby('key')

    #return_dic = {}
    return_df = pd.DataFrame()
    for name, group in libdf:
        try:
            return_df = return_df.append(matchlib2area_one(name,group,areadf.get_group(name),areatype))
            # return_dic[name] = matchlib2area_one(name,group,areadf.get_group(name),areatype)
        except:
            print(name+' did not match successfully')
    return return_df.reset_index()

In [4]:
key = 'FA13.D'
libdf = agi[key].results.lib
ticdf  = agi[key].results.tic
fiddf = agi[key].results.fid
matchlib2area_one('FA14.D',libdf,ticdf,'tic')

,header=,pk,rt,pct_area,library_id,ref,cas,qual,tic_area
0,1=,1,9.8673,1.7321,Methyl myristate,6,000000-00-0,97,251790
1,2=,3,10.9316,13.5256,Methyl palmitoleate,9,000000-00-0,99,1966188
2,3=,4,11.0460,41.6107,Methyl palmitate,10,000000-00-0,97,6048885
3,4=,7,12.1732,29.3428,cis-9-Oleic methyl ester,33,000000-00-0,94,4265513
4,5=,8,12.3220,1.5754,Methyl stearate,19,000000-00-0,98,229017


In [11]:
matched_df = matchlib2area_all(agi,'fid')

FA11.D did not match successfully
FA14_2.D did not match successfully
   index header=   pk      rt  pct_area           library_id   ref  \
0      0      1=  1.0  5.7877    2.0335     Methyl octanoate  17.0   
1      1      2=  2.0  7.3441    3.4015     Methyl decanoate   1.0   
2      2      3=  3.0  8.0364    1.7448   Methyl undecanoate   2.0   
3      3      4=  4.0  8.6715    3.9674   Methyl dodecanoate   3.0   
4      4      5=  5.0  9.2781    1.9607  Methyl tridecanoate   4.0   

           cas  qual     key  fid_area  
0  000000-00-0  96.0  FA03.D  14894660  
1  000000-00-0  98.0  FA03.D  24914490  
2  000000-00-0  98.0  FA03.D  13794040  
3  000000-00-0  98.0  FA03.D  18211710  
4  000000-00-0  99.0  FA03.D  54895790  


FA14_2.D did not match successfully
     index header=    pk       rt  pct_area  \
0        0      1=   1.0   5.7877    2.0335   
1        1      2=   2.0   7.3441    3.4015   
2        2      3=   3.0   8.0364    1.7448   
3        3      4=   4.0   8.6715    3.9674   
4        4      5=   5.0   9.2781    1.9607   
5        5      6=   6.0   9.7930    1.8832   
6        6      7=   7.0   9.8503    4.7016   
7        7      8=   8.0  10.3595    1.9783   
8        8      9=   9.0  10.4168    2.4864   
9        9     10=  10.0  10.9317    2.1450   
10      10     11=  11.0  11.0462    7.4947   
11      11     12=  12.0  11.5840    2.1917   
12      12     13=  13.0  11.6928    1.9137   
13      13     14=  14.0  12.0647    1.9888   
14      14     15=  15.0  12.1791   15.5051   
15      15     16=  16.0  12.3050    5.6958   
16      16     17=  17.0  13.1175    1.8983   
17      17     18=  18.0  13.1576    1.5400   
18      18     19=  19.0  13.1976    2.3509   
19      19     20=  20.0